# Module 6: Grouping for Aggregation, Filtration and Transformation
* [Chap 7 Pandas Cookbook]


## Split-Apply-combine approach

* **Split:** break the dataset into small parts
* **Apply:** Apply a function to each part separately
* **Combine:** Combine the results of the function application back into a dataset

In [1]:
import pandas as pd
import numpy as np

# Defining an aggregation

* **Aggregation:**           Taking many values and converting them into a single value like sum, mean etc
* **Aggregating columns:**  Columns whose values are being aggregated
* **Aggregating functions:** Functions that define how the aggregation takes place (sum, min, max std, etc are examples)
* **Grouping columns:**      One or more columns the combination of whose values define a group of observations (to be aggregated)

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


## Aggregation is done with the groupby function.
### The format is:       
# df.groupby(['list', 'of', 'grouping', 'columns'])
# df.groupby('single_column')

In [3]:
# To groupby function, we can chain the agg function. 
# The agg function can take as argument a dictionary specifying the aggregation column and the aggregation function

flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

# In the above example, the aggregation column is the ARR_DELAY and teh aggregation function is the mean

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [4]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

# The following aaggregating functions may be passsed to .agg or chained directly as a method

* **min**
* **max**
* **mean**
* **median**
* **sum**
* **count**
* **std**
* **size**
* **describe**
* **nunique**
* **idmin**
* **idmax**



## The groupby operation returns a groupby object

In [5]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.groupby.DataFrameGroupBy

In [6]:
# The following command will result in an error because an.sqrt function does not apply
# flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)

# Grouping and aggregating with multiple columns and functions

In [7]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


### Multiple grouping variables

In [8]:
# We can have more than one grouping variables (This is the multiple columns case)
# The number of cancelled flights for every airline, per day weekday
# Here AIRLINE and WEEKDAY are two grouping variables
# The groups will be defined using every unique combination of the values of the two variables

flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum').head(7)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
Name: CANCELLED, dtype: int64

### Multiple aggregating COLUMNS and multiple aggregating FUNCTIONS

In [9]:
# We can also have multiple aggregation functions in groupby operation
# Example: Find the number and percentage of cancelled and diverted flights for every airline per weekday

flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head(7)

#      -AIRLINE and WEEKDAY are grouping variables  
#      -CANCELLED & DIVERTED are aggregating columns
#      -'sum' and 'mean' are aggregation functions passed to .agg method

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
        6              21  0.018667        9  0.008000
        7              29  0.021837        1  0.000753

## Multiple aggregating variables, each with different aggregating functions 
## ==> Use dictionaries

In [14]:
# For each origin to destination flight, find the total number of flights, 
# the number and percentage of cancelled flights and the average and variance of the airtime. 

# How to use Dictionary: For each aggregating variable (key) we can pass on list of aggregating function names (values) 
#                        to be applied on it


group_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED':['sum', 'mean', 'size'], 
            'AIR_TIME':['mean', 'var']}
flights.groupby(group_cols).agg(agg_dict).head()



# Alternatively:
# flights.groupby(['ORG_AIR', 'DEST_AIR']).agg({'CANCELLED': ['sum', 'mean', 'size'], 
#                                               'AIR_TIME':['mean', 'var']}).head()

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

# Removing the MultiIndex after grouping

In [11]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [30]:
# This groupby operation will create a multiIndex

airline_info = flights.groupby(['AIRLINE', 'WEEKDAY'])\
                      .agg({'DIST':['sum', 'mean'], 
                                    'ARR_DELAY':['min', 'max']}).astype(int)

airline_info.head()

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732

### Removing multiIndex is just re-naming the columns 

In [31]:
level0 = airline_info.columns.get_level_values(0)
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [32]:
level1 = airline_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [33]:
# Concatenating level(0) and level(1) column names with '_'

airline_info.columns = level0 + '_' + level1

In [34]:
airline_info.head(7)

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
        6         1265340       1124            -50            858
        7         1461906       1100            -49            626

### .reset_index method -- for removing multiIndex from rows 
### (this will return row labels to single level) 

In [36]:
airline_info.reset_index().head(7)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
5,AA,6,1265340,1124,-50,858
6,AA,7,1461906,1100,-49,626


## The multiIndex can be avoided by setting as-index=False in groupby operation

In [37]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


## The groupby operation sorts rows by default
## To retain the order encountered in the original data, use sort=False

In [38]:
flights.groupby(['AIRLINE'], as_index=False, sort=False)['DIST'].agg('mean')

,AIRLINE,DIST
0,WN,809.985626
1,UA,1230.918891
2,MQ,404.229041
3,AA,1114.347865
4,F9,969.593014
5,EV,460.237453
6,OO,511.239375
7,NK,1047.428100
8,US,1181.226625
9,AS,1065.884115


# Customizing an aggregation function: 
##                write your own aggregation function

In [22]:
college = pd.read_csv('data/college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5
